In [1]:
import geopandas as gpd
from shapely.geometry import Point, MultiPoint
import shapely.speedups
shapely.speedups.enable()

In [2]:
world_filepath = gpd.datasets.get_path('naturalearth_lowres')
world = gpd.read_file(world_filepath)
world.head()

,pop_est,continent,name,iso_a3,gdp_md_est,geometry
0,920938,Oceania,Fiji,FJI,8374.0,"MULTIPOLYGON (((180.00000 -16.06713, 180.00000..."
1,53950935,Africa,Tanzania,TZA,150600.0,"POLYGON ((33.90371 -0.95000, 34.07262 -1.05982..."
2,603253,Africa,W. Sahara,ESH,906.5,"POLYGON ((-8.66559 27.65643, -8.66512 27.58948..."
3,35623680,North America,Canada,CAN,1674000.0,"MULTIPOLYGON (((-122.84000 49.00000, -122.9742..."
4,326625791,North America,United States of America,USA,18560000.0,"MULTIPOLYGON (((-122.84000 49.00000, -120.0000..."


In [9]:
merged= pd.read_csv('merged_v0.23_nocountry.csv', index_col='ID')

/home/allen/Documents/Programs/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (3,4,9,10,12,15,16,18) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [11]:
merged= merged.drop(['Unnamed: 0', 'Unnamed: 0.1'],axis=1)

In [4]:
gfm= pd.read_csv('gfm_v0.1.csv')

In [5]:
len(merged)

167008

In [13]:
gfm.head()

,Unnamed: 0,event_id,location_ID,location_ID_url,name,type,country_location_ID,country_ISO3,start,end,time of detection,lon,lat,country,state
0,0,46467,g-1958070,http://www.geonames.org/1958070,Provinsi Maluku Utara,adm1,g-1643084,IDN,2020-08-26 08:08:17,2020-08-27 15:10:11,2020-08-26 08:19:41,127.84241,0.62621,Indonesia,North Maluku
1,1,46128,g-1996549,http://www.geonames.org/1996549,Provinsi Papua Barat,adm1,g-1643084,IDN,2020-08-16 15:23:38,2020-08-17 10:12:57,2020-08-17 10:12:57,134.06118,-0.86531,Indonesia,West Papua
2,2,38314,g-1996549,http://www.geonames.org/1996549,Provinsi Papua Barat,adm1,g-1643084,IDN,2019-10-17 08:23:35,2019-10-17 09:05:25,2019-10-17 09:05:25,134.06118,-0.86531,Indonesia,West Papua
3,3,45647,g-1996549,http://www.geonames.org/1996549,Provinsi Papua Barat,adm1,g-1643084,IDN,2020-08-01 14:37:12,2020-08-01 15:18:32,2020-08-01 15:18:32,134.06118,-0.86531,Indonesia,West Papua
4,4,34907,g-1996549,http://www.geonames.org/1996549,Provinsi Papua Barat,adm1,g-1643084,IDN,2019-06-08 01:37:03,2019-06-10 14:29:44,2019-06-08 11:54:31,134.06118,-0.86531,Indonesia,West Papua


In [17]:
merged.tail()

,DATE_BEGIN,DATE_END,DURATION,LON,LAT,COUNTRY,STATE,LOCATION,AREA,FATALITY,DAMAGE,SEVERITY,SOURCE,CAUSE,SOURCE_DB,SOURCE_ID,DESCRIPTION
ID,,,,,,,,,,,,,,,,,
167003,20170406232000,NaN,NaN,-71.447887,42.711457,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mPing,NaN,River/Creek overflowing; Cropland/Yard/Basemen...
167004,20170407000820,NaN,NaN,-76.178844,42.087840,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mPing,NaN,River/Creek overflowing; Cropland/Yard/Basemen...
167005,20170407000824,NaN,NaN,-76.194438,42.083504,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mPing,NaN,River/Creek overflowing; Cropland/Yard/Basemen...
167006,20170407001120,NaN,NaN,-76.229550,42.091994,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mPing,NaN,Street/road flooding; Street/road closed; Vehi...
167007,20170407001547,NaN,NaN,-76.250373,42.108731,USA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,mPing,NaN,Street/road flooding; Street/road closed; Vehi...


In [24]:
merged.loc[167008:167008+48051, 'SOURCE_DB']= 'GFM'

In [29]:
merged= merged.reindex(np.arange(167008+48051))

In [34]:
merged.loc[167008:167008+48051, 'LON']= gfm.lon.values
merged.loc[167008:167008+48051, 'LAT']= gfm.lat.values
merged.loc[167008:167008+48051, 'DATE_BEGIN']= pd.to_datetime(gfm.end).dt.strftime('%Y%m%d%H%M%S').tolist()
merged.loc[167008:167008+48051, 'DATE_END']= pd.to_datetime(gfm.end).dt.strftime('%Y%m%d%H%M%S').tolist()
merged.loc[167008:167008+48051, 'DURATION']= (pd.to_datetime(gfm.start) - pd.to_datetime(gfm.end)).dt.days.values
merged.loc[167008:167008+48051, 'COUNTRY']= gfm.country.tolist()
merged.loc[167008:167008+48051, 'STATE']= gfm.state.tolist()
merged.loc[167008:167008+48051, 'SOURCE_ID']= gfm.event_id.values
merged.loc[167008:167008+48051, 'SOURCE_DB']= 'GFM'

In [39]:
gdf = gpd.GeoDataFrame(
    merged, geometry=gpd.points_from_xy(merged.LON, merged.LAT))

In [43]:
def classCountry(feature):
    masks= gdf.within(feature.geometry)
    return gdf[masks].index.values

In [44]:
IDs= world.apply(classCountry, axis=1)

In [48]:
for i,ids in enumerate(IDs):
    merged.loc[ids, 'COUNTRY']= world.loc[i, 'name']

In [47]:
merged.loc[:,'COUNTRY']=''

In [49]:
merged.COUNTRY.unique()

array(['United States of America', '', 'France', 'United Kingdom',
       'Ireland', 'Ecuador', 'Puerto Rico', 'Australia', 'Peru',
       'Colombia', 'Bolivia', 'Chile', 'Mexico', 'Philippines', 'Canada',
       'Georgia', 'Mali', 'Argentina', 'Dominican Rep.', 'Bulgaria',
       'Serbia', 'China', 'Czechia', 'Costa Rica', 'Spain', 'Germany',
       'Brazil', 'Italy', 'Netherlands', 'Uruguay', 'Poland', 'Morocco',
       'Vietnam', 'Romania', 'Tanzania', 'South Korea', 'Cuba',
       'Malaysia', 'Denmark', 'Nicaragua', 'Algeria', 'Indonesia',
       'Mozambique', 'New Zealand', 'Bangladesh', 'India', 'Chad',
       'Sudan', 'Nepal', 'Austria', 'Ethiopia', 'Japan', 'Sweden',
       'Haiti', 'Nigeria', 'Benin', 'Thailand', 'Jamaica', 'Venezuela',
       'Iran', 'Tunisia', 'Madagascar', 'Afghanistan', 'South Africa',
       'Egypt', 'Taiwan', 'Botswana', 'Turkey', 'Kenya', 'Uganda',
       'Sri Lanka', 'Ukraine', 'Russia', 'Pakistan', 'Burkina Faso',
       'Niger', 'El Salvador', 'Seneg

In [38]:
merged.loc[merged.COUNTRY=='USA', 'COUNTRY']= 'United States of America'
merged.loc[merged.COUNTRY=='UK', 'COUNTRY']= 'United Kindom'

In [50]:
merged.to_csv('merged_v0.30.csv')

In [64]:
merged.drop(['DESCRIPTION'], axis=1)[(~pd.isna(merged.LON)) & (~pd.isna(merged.LAT))].to_csv('merged_v0.23_GEE.csv')

In [65]:
gdf = gpd.GeoDataFrame(
    merged, geometry=gpd.points_from_xy(merged.LON, merged.LAT))

In [66]:
gdf.drop(['DESCRIPTION'], axis=1)[(~pd.isna(gdf.LON)) & (~pd.isna(gdf.LAT))].to_file('GFD_v0.23_GEE')

In [42]:
gdf.to_file('merged_v0.23.shp')

In [ ]:
aoi= gpd.read_file('/home/allen/Documents/Projects/Harvey/excessive_rain/subbasins/08076700.shp')

## V0.45, refill country based on country abbre

In [2]:
df= pd.read_csv('merged_v0.45.csv', index_col='ID')

/home/allen/Documents/Programs/miniconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3147: DtypeWarning: Columns (1,2,8,13,14,16,17) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [25]:
yearcodes= df.loc[df.SOURCE_DB=='EM-DAT', 'SOURCE_ID'].str.split('-')

In [51]:
codes= pd.read_csv('country_codes.txt',delimiter='\t', header=None)
codes['code']= codes[0].str.split(' ',n=1, expand=True)[0]
codes['name']= codes[0].str.split(' ',n=1, expand=True)[1]

In [98]:
def decode_country(code):
    if code not in codes.code.tolist():
        print(code +' not found in this dataset!!!')
    else:
        country= codes.loc[codes.code==code,'name'].values[0]
        return country

In [70]:
codes[codes.code=='DFR']

,0,code,name


In [100]:
codes.loc[245, 'code']='DFR'
codes.loc[245, 'name']='Netherlands'
codes.loc[246, 'code']='YUG'
codes.loc[246, 'name']='Serbia'
codes.loc[247, 'code']='YMD'
codes.loc[247, 'name']='Yemen'
codes.loc[248, 'code']='YMN'
codes.loc[248, 'name']='Yemen'

codes.loc[249, 'code']='CSK'
codes.loc[249, 'name']='Slovenia'

codes.loc[250, 'code']='SUN'
codes.loc[250, 'name']='Russia'

codes.loc[251, 'code']='SCG'
codes.loc[251, 'name']='Serbia'

codes.loc[252, 'code']='ZWE'
codes.loc[252, 'name']='Zimbabwe'

codes.loc[253, 'code']='SSD'
codes.loc[253, 'name']='Sudan'

In [101]:
for i in range(len(yearcodes)):
    ind= yearcodes.index[i]
    df.loc[ind,'COUNTRY']= decode_country(yearcodes[ind][-1])
    if pd.isna(df.loc[ind,'DATE_BEGIN']):
        df.loc[ind,'DATE_BEGIN']=yearcodes[ind][0]

SPI not found in this dataset!!!
SPI not found in this dataset!!!


In [103]:
sorted(df.COUNTRY.unique()[~pd.isna(df.COUNTRY.unique())])

['Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Angola',
 'Anguilla',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bolivia, Plurinational State of',
 'Bosnia and Herz.',
 'Bosnia and Herzegovina',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Central African Rep.',
 'Central African Republic',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Congo, the Democratic Republic of the',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czech Republic',
 'Czechia',
 "Côte d'Ivoire",
 'Dem. Rep. Congo',
 'Denmark',
 'Djibouti',
 'Dominican Rep.',
 'Dominican Republic',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Eritrea',
 'Estonia',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'French Guiana',
 'French Polynesia',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germa

In [104]:
# Unify country names
df.loc[df.COUNTRY=='Bolivia, Plurinational State of', 'COUNTRY']='Bolivia'
df.loc[df.COUNTRY=='Bosnia and Herzegovina', 'COUNTRY']='Bosnia and Herz.'
df.loc[df.COUNTRY=='Central African Republic', 'COUNTRY']='Central African Rep.'
df.loc[df.COUNTRY=='Congo, the Democratic Republic of the', 'COUNTRY']='Dem. Rep. Congo'
df.loc[df.COUNTRY=='Czech Republic', 'COUNTRY']='Czechia'
df.loc[df.COUNTRY=='Dominican Republic', 'COUNTRY']='Dominican Rep.'
df.loc[df.COUNTRY=='Hong Kong', 'COUNTRY']='China'
df.loc[df.COUNTRY=='Iran, Islamic Republic of', 'COUNTRY']='Iran'
df.loc[df.COUNTRY=="Korea, Democratic People's Republic of", 'COUNTRY']='South Korea'
df.loc[df.COUNTRY=="Korea, Republic of", 'COUNTRY']='North Korea'
df.loc[df.COUNTRY=="Lao People's Democratic Republic", 'COUNTRY']='Laos'
df.loc[df.COUNTRY=="Libyan Arab Jamahiriya", 'COUNTRY']='Libya'
df.loc[df.COUNTRY=="Macedonia, the former Yugoslav Republic of", 'COUNTRY']='Macedonia'
df.loc[df.COUNTRY=="Moldova, Republic of", 'COUNTRY']='Moldova'
df.loc[df.COUNTRY=="Palestinian Territory, Occupied", 'COUNTRY']='Palestine'
df.loc[df.COUNTRY=="Russian Federation", 'COUNTRY']='Russia'
df.loc[df.COUNTRY=="Solomon Islands", 'COUNTRY']='Solomon Is.'
df.loc[df.COUNTRY=="Taiwan", 'COUNTRY']='China'
df.loc[df.COUNTRY=="Taiwan, Province of China", 'COUNTRY']='China'
df.loc[df.COUNTRY=="Tanzania, United Republic of", 'COUNTRY']='Tanzania'
df.loc[df.COUNTRY=="United States", 'COUNTRY']='United States of America'
df.loc[df.COUNTRY=="Venezuela, Bolivarian Republic of", 'COUNTRY']='Venezuela'
df.loc[df.COUNTRY=="Viet Nam", 'COUNTRY']='Vietnam'



In [105]:
sorted(df.COUNTRY.unique()[~pd.isna(df.COUNTRY.unique())])

['Afghanistan',
 'Albania',
 'Algeria',
 'American Samoa',
 'Angola',
 'Anguilla',
 'Argentina',
 'Armenia',
 'Australia',
 'Austria',
 'Azerbaijan',
 'Bahamas',
 'Bangladesh',
 'Barbados',
 'Belarus',
 'Belgium',
 'Belize',
 'Benin',
 'Bhutan',
 'Bolivia',
 'Bosnia and Herz.',
 'Botswana',
 'Brazil',
 'Brunei',
 'Bulgaria',
 'Burkina Faso',
 'Burundi',
 'Cambodia',
 'Cameroon',
 'Canada',
 'Cape Verde',
 'Central African Rep.',
 'Chad',
 'Chile',
 'China',
 'Colombia',
 'Comoros',
 'Congo',
 'Costa Rica',
 'Croatia',
 'Cuba',
 'Cyprus',
 'Czechia',
 "Côte d'Ivoire",
 'Dem. Rep. Congo',
 'Denmark',
 'Djibouti',
 'Dominican Rep.',
 'Ecuador',
 'Egypt',
 'El Salvador',
 'Eritrea',
 'Estonia',
 'Ethiopia',
 'Fiji',
 'Finland',
 'France',
 'French Guiana',
 'French Polynesia',
 'Gabon',
 'Gambia',
 'Georgia',
 'Germany',
 'Ghana',
 'Greece',
 'Grenada',
 'Guadeloupe',
 'Guatemala',
 'Guinea',
 'Guinea-Bissau',
 'Guyana',
 'Haiti',
 'Honduras',
 'Hungary',
 'Iceland',
 'India',
 'Indonesia'

In [106]:
df.to_csv('merged_v0.46.csv')